In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Practice").getOrCreate()
df_pyspark = spark.read.csv('test4.csv', header = True, inferSchema = True)

In [5]:
df_pyspark.show()

+---------+---+----------+------+
|     Name|Age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [7]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
#clubbing features: Age and Experience together into one 'features' column.

from pyspark.ml.feature import VectorAssembler

featureassembler = VectorAssembler(inputCols = ['Age', 'Experience'], outputCol = 'features')
output = featureassembler.transform(df_pyspark)

In [10]:
output.show()

+---------+---+----------+------+-----------+
|     Name|Age|Experience|Salary|   features|
+---------+---+----------+------+-----------+
|    Krish| 31|        10| 30000|[31.0,10.0]|
|Sudhanshu| 30|         8| 25000| [30.0,8.0]|
|    Sunny| 29|         4| 20000| [29.0,4.0]|
|     Paul| 24|         3| 20000| [24.0,3.0]|
|   Harsha| 21|         1| 15000| [21.0,1.0]|
|  Shubham| 23|         2| 18000| [23.0,2.0]|
+---------+---+----------+------+-----------+



In [12]:
#extracting useful data columns: features and salary

finalised_data = output.select('features', 'Salary')
finalised_data.show()

+-----------+------+
|   features|Salary|
+-----------+------+
|[31.0,10.0]| 30000|
| [30.0,8.0]| 25000|
| [29.0,4.0]| 20000|
| [24.0,3.0]| 20000|
| [21.0,1.0]| 15000|
| [23.0,2.0]| 18000|
+-----------+------+



In [99]:
#importing the machine learning functions

from pyexpat import features
from pyspark.ml.regression import LinearRegression

#train-test split
train_data, test_data = finalised_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol = 'features', labelCol = 'Salary')

regressor = regressor.fit(train_data)
print(regressor.coefficients)
print(regressor.intercept)


[-176.33410672854762,1320.1856148492002]
19812.064965197405


In [100]:
pred_results = regressor.evaluate(test_data)

In [101]:
pred_results.predictions.show()

+-----------+------+------------------+
|   features|Salary|        prediction|
+-----------+------+------------------+
| [21.0,1.0]| 15000|17429.234338747105|
|[31.0,10.0]| 30000| 27547.56380510443|
+-----------+------+------------------+



In [102]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError, pred_results.rootMeanSquaredError

(2440.8352668213374, 5957811.381290973, 2440.862835411071)